<a href="https://colab.research.google.com/github/VaibhavNagrale007/Adversarial-Machine-Learning-BTP/blob/main/Sem%207/Red_Attack_Xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prepare the Data Loaders:

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Specify the path to your dataset
dataset_path = '/content/drive/MyDrive/Dataset/xray_pneumonia'

# Set up data generators
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

train_generator = datagen.flow_from_directory(
    directory=dataset_path + '/train',
    target_size=(30, 30),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

test_generator = datagen.flow_from_directory(
    directory=dataset_path + '/test',
    target_size=(30, 30),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 5546 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


Build a New Model:

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import save_model, load_model

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(30, 30, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming your training steps per epoch is the number of training samples divided by the batch size
steps_per_epoch = len(train_generator) // batch_size

# Train the model
model.fit(train_generator, epochs=15, steps_per_epoch=steps_per_epoch)

# Save the model
model.save('/content/drive/MyDrive/Dataset/pmodel-Xray.h5')

Epoch 1/15
5/5 [==============================] - 65s 13s/step - loss: 0.6250 - accuracy: 0.6438
Epoch 2/15
5/5 [==============================] - 57s 12s/step - loss: 0.6104 - accuracy: 0.6812
Epoch 3/15
5/5 [==============================] - 61s 12s/step - loss: 0.5817 - accuracy: 0.6750
Epoch 4/15
5/5 [==============================] - 58s 11s/step - loss: 0.5152 - accuracy: 0.7250
Epoch 5/15
5/5 [==============================] - 59s 12s/step - loss: 0.5118 - accuracy: 0.7937
Epoch 6/15
5/5 [==============================] - 57s 12s/step - loss: 0.4053 - accuracy: 0.8313
Epoch 7/15
5/5 [==============================] - 61s 12s/step - loss: 0.3691 - accuracy: 0.8313
Epoch 8/15
5/5 [==============================] - 51s 10s/step - loss: 0.3445 - accuracy: 0.8687
Epoch 9/15
5/5 [==============================] - 52s 10s/step - loss: 0.3355 - accuracy: 0.8625
Epoch 10/15
5/5 [==============================] - 51s 10s/step - loss: 0.3571 - accuracy: 0.8313
Epoch 11/15
5/5 [============

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Load existing Model:

In [ ]:
from tensorflow.keras.models import load_model

model_path = '/content/drive/MyDrive/Dataset/pmodel-Xray.h5'
model = load_model(model_path)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Evaluate on Test Data:

In [ ]:
# Evaluate on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


20/20 [==============================] - 114s 6s/step - loss: 0.3822 - accuracy: 0.8157
Test Accuracy: 81.57%


Code

In [ ]:
import cv2
import numpy as np
from PIL import Image

height = 30
width = 30

def max_diff(img1,img2):
    img = img1 - img2
    return np.amax(img)

def pred(image):
    data = []
    data.append(image)
    X_test = np.array(data)
    X_test = X_test.astype('float32')/255
    X_test = X_test.reshape(1,height,width,3)
    predict_x=model.predict(X_test)
    pred_target_image = 1 if predict_x[0, 0] > 0.5 else 0
    return pred_target_image

def boundary_estimation(source, target, dmin):
    Ii = ((source + target)/2.0)
    k = pred(Ii)
    delta = max_diff(source, Ii)
    Ia2 = source
    Ib2 = target
    p = Ib2
    while (delta > dmin):
        if (pred(Ia2) != k):
            Ib2 = Ii
        else:
            Ia2 = Ii
        Ii = ((Ia2+Ib2)/2.0)
        k = pred(Ii)
        delta = max_diff(Ia2,Ii)
    return Ii

def go_out(source,iout,alpha):
    i_diff = iout - source
    pred_source = pred(source)
    inew = iout
    # Note: added limit to end
    limit = 10
    while (pred(inew)==pred_source):
        inew = inew + alpha*(i_diff)
        limit = limit - 1
        if (limit < 0): break

    return inew

In [ ]:
source_image_path = "/content/drive/MyDrive/Dataset/xray_pneumonia/train/NORMAL/IM-0158-0001.jpeg";
target_image_path = "/content/drive/MyDrive/Dataset/xray_pneumonia/train/PNEUMONIA/person120_virus_226.jpeg";

img = (np.asarray(Image.open(source_image_path)))
img2_rgb = np.repeat(img[:, :, np.newaxis], 3, axis=2)
image_from_array = Image.fromarray(img2_rgb, 'RGB')
img1 = image_from_array.resize((height, width))
img1=np.array(img1)
img1=img1.reshape(height,width,3)
source_image = np.array(img1)

img = (np.asarray(Image.open(target_image_path)))
img2_rgb = np.repeat(img[:, :, np.newaxis], 3, axis=2)
image_from_array = Image.fromarray(img2_rgb, 'RGB')
img2 = image_from_array.resize((height, width))
img2=np.array(img2)
img2=img2.reshape(height,width,3)
target_image = np.array(img2)

i = boundary_estimation(source_image,target_image,1.0)

1/1 [==============================] - 0s 21ms/step


In [ ]:
print (pred(i))
print (pred(source_image)) #Class of source image
print (pred(target_image)) #Class of target image

1/1 [==============================] - 0s 25ms/step
1
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 29ms/step
1


In [ ]:
ii = go_out(source_image,i,0.01)

1/1 [==============================] - 0s 33ms/step


In [ ]:
# pred(ii)
print(pred(ii))

1/1 [==============================] - 0s 33ms/step
1


In [ ]:
def array_diff(d1):
    sumd1 = 0.0
    for i in range(0,3):
        for j in range(0,height):
            for k in range(0,width):
                d1[j][k][i] = d1[j][k][i]*d1[j][k][i]
                sumd1 = sumd1 + d1[j][k][i]
    return (sumd1)

def gradient_estimation(source, target, adversarial, n, theta):
    Ia = source
    Ib = target
    Ii = adversarial
    Io = np.zeros((2700))
    X = np.random.randint(0,2700, size=n)
    for i in X:
        Io[i] = 255
    Io = Io.reshape((height,width,3))
    Ii2 = Ii + theta*Io
    Ii2_new = boundary_estimation(Ia, Ii2, 1.0)
    Ii2_new = go_out(source,Ii2_new,0.01)
    diff2 = Ii2_new - Ia
    diff1 = Ii - Ia
    d2 = array_diff(diff2)
    d1 = array_diff(diff1)
    if (d2 > d1):
        return (-1, Ii2_new)
    elif (d1 > d2):
        return (1, Ii2_new)
    else:
        return (0,Ii2_new)

def efficient_update(source, target, adversarial, I2, g, j):
    Ia = source
    Ib = target
    Ii = adversarial
    Ii2 = I2
    delta = g*(Ii2 - Ii)
    l = j
    Inew = Ii + l*delta

    diff1 = Inew - Ia
    diff2 = Ii - Ia
    d1 = array_diff(diff1)
    d2 = array_diff(diff2)
    ii = 0
    it = 0
    while(d1 > d2):
        l = (l/2.0)
        Inew = Ii + l*delta
        if(pred(Inew)==pred(source)):
            Inew = go_out(source,Inew,0.01)
        it = it + 1
        d1 = array_diff(Inew-Ia)
        if(it>100):
            break
    if (d1 > d2):
        print(ii)
        ii = ii + 1
        Inew = Ii
    return Inew

def iteration(itr, source, target, n, theta, j, dmin):
    targett = target
    sourcee = source
    for i in range(itr):
        print ("\n Iteration: ",i)
        adversarial_image = boundary_estimation(sourcee, targett, dmin)
        adversarial_image = go_out(sourcee,adversarial_image,0.01)
        (g, Iii2) = gradient_estimation(sourcee, targett, targett, n, theta)
        targett = efficient_update(sourcee, targett, adversarial_image, Iii2, g, j)
        if (pred(targett) == pred(source)):
            j = j/2.0
        fin = targett
        if(pred(targett)==pred(sourcee)):
            fin = go_out(sourcee,targett,0.01)
        if(array_diff(fin-sourcee)<array_diff(adversarial_image-sourcee)):
            targett = fin

    return fin

# Note: changed 1000 -> 1
final = iteration(2,source_image,target_image,5,0.196,5.0,1.0)

pred(final)

Image.fromarray(source_image.astype('uint8')).save('/content/drive/MyDrive/Dataset/original_xray3.png')
Image.fromarray(final.astype('uint8')).save('/content/drive/MyDrive/Dataset/perturbed_xray3.png')
# s = measure.compare_ssim(arr[1],arr[0])

original = cv2.imread("/content/drive/MyDrive/Dataset/original_xray3.png")
perturb = cv2.imread("/content/drive/MyDrive/Dataset/perturbed_xray3.png")

#s = measure.compare_ssim(original,perturb,multichannel=True)
#print(s)
print(perturb)
print(original)


 Iteration:  0
1/1 [==============================] - 0s 22ms/step


<ipython-input-53-722d0a678183>:6: RuntimeWarning: overflow encountered in ubyte_scalars
  d1[j][k][i] = d1[j][k][i]*d1[j][k][i]


1/1 [==============================] - 0s 27ms/step

 Iteration:  1
1/1 [==============================] - 0s 35ms/step
0
1/1 [==============================] - 0s 35ms/step
[[[ 40  40  40]
  [ 45  45  45]
  [ 38  38  38]
  ...
  [ 55  55  55]
  [ 25  25  25]
  [ 24  24  24]]

 [[ 65  65  65]
  [111 111 111]
  [100 100 100]
  ...
  [ 15  15  15]
  [  4   4   4]
  [ 15  15  15]]

 [[ 65  65  65]
  [118 118 118]
  [117 117 117]
  ...
  [ 23  23  23]
  [255 255 255]
  [  9   9   9]]

 ...

 [[  0   0   0]
  [  2   2   2]
  [ 83  83  83]
  ...
  [ 97  97  97]
  [ 84  84  84]
  [  2   2   2]]

 [[  0   0   0]
  [  3   3   3]
  [ 78  78  78]
  ...
  [ 92  92  92]
  [ 85  85  85]
  [  5   5   5]]

 [[254 254 254]
  [  1   1   1]
  [ 69  69  69]
  ...
  [145 145 145]
  [ 81  81  81]
  [ 10  10  10]]]
[[[ 50  50  50]
  [ 62  62  62]
  [ 61  61  61]
  ...
  [ 72  72  72]
  [ 41  41  41]
  [ 35  35  35]]

 [[ 72  72  72]
  [ 87  87  87]
  [ 81  81  81]
  ...
  [ 37  37  37]
  [ 23  23  23]
  [ 26